# Long Term Potentials

## Marketing

**Marketing Tools**

 - Move registration email signup to Mailchimp from Zoho
 - Why do we use Blogger for the blog? Could we swap it out for Ghost without breaking anything? 
 - Script to accept Positionly data as CSV, remove lines with >100 cells in (no mentions), aggregate and graph results over time.

**Blog Posts**

 - Digital nomadism
 - Evaluation of cloud service providers and migration from Liberata to (?)cloud

**Cross-Selling**

Once an auction is over, we should have popups within the app which point Hosts to SIM Dojo and Innovation Dojo for their onboarding and supplier relations needs. Currently there is no through route; there should be far fewer seams and far more encouragement to have hosts make use of all the components of the Dojo.

**Naming**

 - Suggest 'Hombu Dojo' as name for the office.


## Data Management

**Redis**

For caching, session management and as a pub/sub server to replace the existing Socket.io situation. As an in-memory db it permits more aggressive caching than we could otherwise require, so may improve performance relative to the MySQL setup. New set of instances could include a dedicated or composed Redis instance.
  
**Databases**

Currently MySQL; if the throughput is significant might look at Aurora as an alternative (if on AWS)

**Read Replicas**

Read Replica of production DB on separate server for testing purposes. Possible to do this automatically once moved to AWS or Google Cloud; would allow full separation of MarketDojo and QueryDojo and also give us a rolling backup. It would mean that running queries on QD (either by ourselves or through client APIs) would not represent overhead on production server(s)

## Testing

**Test Conversion**

A number of the tests for the app are still in Test::Unit rather than Rspec, and those that are in Rspec are purely for the Ruby. Converting all the Ruby tests to Rspec, and then creating a suite of tests for our JS with Jasmine (which, according to GitHub, is actually larger than the Ruby code!) which Rspec runs when commanded, should give us much better clarity on exactly what is working and what is not. Might even uncover some bugs. 

We also have some issues (many issues!) with running tests late at night and early in the morning. Optimally these would be fixed; it appears to be an issue with Timecop. Rspec also generates a large number of failures when running in parallel; even with the necessary multiple test databases and environment variables. This means we have inter-test dependencies, which need to be addressed for true unit testing to work. We should be running all tests in parallel and in randomised order.

**Spec Suites**

Switch from having one main test suite to suites for each [module]? What exactly constitutes a module would need deciding. Migrate all tests from MiniTest to Rspec.

**Coverage Testing**

Add a coverage reporter to both the standard test suite and the Jenkins output to allow us to get a handle on how our coverage is changing over time, and identify those areas where more or better specs are needed.

**Performance Metrics**

Automated testing of pageload times for the site.

## Tooling

**PM Tools**

Moving from Zoho bugtracker to Github issues and Kanban boards

First step: write script to wire Octocat and Zoho API together with correct keys; copy all open bugs younger than January 2015 over to Github issues, with details and relevant tags; easier to keep track of everything in one place. This would also allow planning via GitHub Kanban board, which can be linked directly to issues, commits, releases and milestones. Currently trying to apply internal tickets (Rubocop etc) to GitHub in an effort to test the system.

**Dev Tools**

Foreman to include the pub/sub server in the repo, although it's a separate microservice? 
Webpack if we want to streamline our JS?

Possible slow move to ReactRB/Hyperloop stack, with concomitant reduction in use of JS loops and redirects; it'd be nice to remove a few thousand lines from the codebase! Nice thing about Component architecture is that we wouldn't have to do it all at once. PolymerComponents or an Elm frontend would also be an option, though neither of those has the advantage of being essentially all Ruby on the front end as well as the back end.

**TypeScript**

We have exactly one CoffeeScript file, a bunch of jQuery files and some plain JS... plus some inline JS in some of the views.

Good practice would be to remove the latter entirely.

The jQuery, CoffeeScript and JS we could almost certainly move to TypeScript. It would be quite a significant effort, but we would be able to compress and typecheck everything; it should also reduce runtime errors (by making them compile-time errors).

## Code Quality and Tech Debt

**Rubocopping**

The automated stuff we can do immediately, that's not a problem, although some of the automatic cops aren't very bright and break things. Once that's done, though, the rest need manually changing. This is especially important if we want to upgrade the Rails version, since that's going to break many, many things further down the line and it's hard to tell what the real problem is if some of it is syntax errors. See also **deprecations**.

**Jenkins Code Quality Tools**

Add numeric output for automated `rubocop`, `rspec` and `reek` testing to Jenkins; try to avoid having tens of thousands of warnings by having a human look at those. Automated filters can only take us so far. Metric_Fu is useful but only creates the report, it doesn't actually create warnings or email us anything.

**TODO Cleanup**

Codebase as a whole is short on comments. Some use of TODO, a couple of HACKs, almost no NOTEs. No comment attributions **or dating** even for TODOs. Trivial comments e.g. "My bad coding :-(" have been copy/paste replicated in various places.

Find all TODO, NOTE etc comments (including those which should be but aren't!), check if they have been assigned issues on the issue tracker, date and attribute them if possible. If they're redundant remove them, if they indicate problems but don't have detail, add details.

**Documentation**

Follows on from *TODO Cleanup*. Connections between .erb templates, .js scripts and .rb backend should be better (or at all!) documented. Current situation requires searching the entire .js corpus for the id or class attribute of the element that one is looking at, evaluating which of the results looks likely, and then changing small pieces to see what happens. 

Gems are also a concern, especially where we are using forks or direct GitHub links. The Gemfile should contain explanations for each gem, what it is for, and where it is a deprecated gem details of when or under what circumstances it can be removed.

**Address Issue of Documentation Review Periods**

Documentation (especially the wiki) should be evaluated at the beginning of each quarter, and where necessary updated.

Dependencies (such as gems, especially those with native dependencies such as Nokogiri) should be evaluated and updated (if necessary and possible given compatibility constraints) at the beginning of each half.

Core modules (Rails in particular) should be evaluated and updated at each minor (x.**X**.x.x) and major (**X**.x.x.x) release, to prevent multi-step upgrades from being necessary in future. These should also be used as opportunities to remove external gems if the functionality is available in Rails.

## Deprecations

**Rspec Deprecations**

Long term because this is going to be a big job. We need to sort out the deprecation issues, whether that's changing some syntax or upgrading some gems. They're slowing us down, not to mention driving the devs insane because running tests generates a sea of rubbish. They also may partly be behind why running Rspec in parallel fails. 

**Ruby Deprecations**

Disclosed by Rspec, Test::Unit and various other tools. Same issue.

**Gem Deprecations**

These will be harder to find, and some may only appear once we upgrade our Rails version and discover old gems don't work. Comatose in particular is definitely end-of-life software, but there may well be others.

Related issue is that Gemfile.lock is holding the app together; running `bundle update` fails in many places due to incompatibilities and pinned gems. 

**Restful Authentication**

Move from Restful Authentication to Devise
Add OAuth (From whom? Possibly partner companies.)

**Summary**

We should investigate all deprecation warnings produced by the various runners. Those which can be fixed before moving up a Rails or Ruby version, fix; that will make the upgrade process considerably easier. Those which require the replacement or upgrade of gems should likewise be addressed where possible.

## Operations Automation and Improvements

**Simian Army**

If we're bringing on bigger clients, and we need to provide things like APIs and service assurances, we will need to (at minimum) mirror the application. No need to shard the database unless we get significantly more business, but the app itself ought to have multiple instances and load balancing. Doing this with Kubernetes (or Amazon's equivalent containerised EC2 service) makes the most sense (see Amazon Scaleability), and once we get a good load balancer in front of it should make the app much more resilient.

As part of this, it would be a good idea to start introducing Simian Army. Janitor Monkey in particular would be helpful; Conformity Monkey is essentially obviated by the use of containerised environments and we don't have the staff to cope with Chaos Monkey yet (although that would be a good plan for the longer term).

**Automatic Scaling**

If we're expecting increases in usage of the app, and in particular intermittent high demand with periods of low use, it makes sense to have a skeleton app running full time which spins up larger and more capable servers for short periods when it is under high load or expects to be due to the number of auctions due to be run that day.

**Docker, Vagrant or Puppet**

Our testing, staging and deploying process is getting unwieldy. Either we need to work something out with Puppet and Vagrant or we need to Dockerise the application. Either way, a fast and replicable way to spin up new servers is going to be essential. See *automatic scaling*.

**Cloud Services**

Amazon provide both database instances (RDS) and compute (EC2), plus storage (S3). Conveniently packaged as Elastic Beanstalk this provides us somewhere to put the app which will autoscale (to a point). As a place to move away from Liberata this offers significant potential.

Equivalent services from IBM Softlayer/Bluemix and Google Cloud are also excellent; investigation is important as it might come down just to price but there may be other factors to consider.

**Automated Deploy**

CodePipeline or similar - reads from GitHub, runs tests etc and integrates directly with related instances. All major suppliers have an option which works in a similar way. Would replace Jenkins, and allow us to move back to continuous integration. This might make things harder or easier on the marketing guys depending on how we approached it.

**Server Upgrades**

Change server to Puma and web server to Caddy for speed and ease of use. In particular Caddy takes care of SSL, fixes for Logjam and similar, and HTTP/2 in perpetuity, with much less faff or configuration than Nginx.

## Client Integration

**Client Data API**

This is a requested feature. Ties into *QueryDojo* with respect to the separate server and read-only database access. Using the Rails 5 API improvements this could actually be a subfunction of QD itself if properly configured, giving us full isolation.

**API for auction creation**

Requires write access which is dangerous to have on an API, but would allow clients to build or configure their own front ends to send JSON packages to our server. Might tie into OAuth.

**User Licence Data Tracking**

 - User licence data in database
  - Ending dates
  - Invoiced?
  - Paid?

## New Product Areas

**Contract Creation, Storage and Blockchain Contracts**

Durable, maximum-trust 'smart contracts' are becoming more popular, underpinned by blockchain technology which makes all of them available and unmodified in perpetuity. Allows for cryptographically verified virtual 'signatures', full edit history and embedded business logic in a fully open and verifiable way. Contracts which (sort of) enforce themselves!

Some clients have already expressed an interest in us having or creating a contracts area. It would be interesting and valuable to the clients if we can build it with this sort of scaleability and security in mind.

## 'Big Data' and Machine Learning applications

**Tensorflow**

We're gathering quite a lot of data about how people bid, the end values of lots given the qualification price, average bid rate, optimum lot length and various other metrics, purely by virtue of retaining all the events in the database.

I think there's an opportunity here to give advice to the Hosts about how they should structure their auctions given their requirements. Exactly how we go about that is a question of its own, but given some specific questions ("what lot length results in the most bids without long waits at the beginning or end, and without cutting off a good competition?" would be a naive example) we could train a TensorFlow neural net on the data we already have and see if it produces good predictions which we could then feed back to the Hosts and help them design their lots better.

**Natural Language Processing**

It would seem reasonable that, given we already provide document uploading capabilities, we could wire the document retrieval to the Google Natural Language API. That would allow all participants and hosts to submit and retrieve documents in their own language, without worrying about to whom they will be sent or in what language they were originally. For companies like Aggreko, who could have someone speaking one language submitting jobs to be bid for in countries where the official language is something different, this would be a significant reduction in friction. Currently the most 'production ready' languages are English and Spanish, which would be somewhat useful in Europe and moreso in the Americas.

In this vein we should encourage the people who spend most time on support (Lewis, Pete, Craig) to use the web interface for support, as Zoho provide a realtime translation service. It looks good to look multinational and talk to people in their own languages.